# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from random import sample

import re

import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

# Import original data and preprocessed data

In [3]:
#sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
#calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

In [4]:
sales_train_val = pd.read_csv(r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\M5 Forecasting\Estimate the unit sales of Walmart retail goods\m5-forecasting-accuracy_unzip files\sales_train_validation.csv')
calendar = pd.read_csv(r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\M5 Forecasting\Estimate the unit sales of Walmart retail goods\m5-forecasting-accuracy_unzip files\calendar.csv')

**Note: This Step is for the extraction of California State and Foods category**

In [5]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOUSEHOLD')]

In [6]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
565,HOUSEHOLD_1_001_CA_1_validation,HOUSEHOLD_1_001,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,3,0,0,0,...,1,0,1,1,1,0,0,0,0,1
566,HOUSEHOLD_1_002_CA_1_validation,HOUSEHOLD_1_002,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,1,1,2,4,...,0,0,0,0,0,1,0,0,0,1
567,HOUSEHOLD_1_003_CA_1_validation,HOUSEHOLD_1_003,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,1,2,1,0,0,0,1,0
568,HOUSEHOLD_1_004_CA_1_validation,HOUSEHOLD_1_004,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,4,0,8,11,...,0,5,0,0,0,0,0,0,1,0
569,HOUSEHOLD_1_005_CA_1_validation,HOUSEHOLD_1_005,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,3,2,2,...,1,0,3,1,2,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10754,HOUSEHOLD_2_512_CA_4_validation,HOUSEHOLD_2_512,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,0,0,2,1,0,0,0,0
10755,HOUSEHOLD_2_513_CA_4_validation,HOUSEHOLD_2_513,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
10756,HOUSEHOLD_2_514_CA_4_validation,HOUSEHOLD_2_514,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,2,0,1,...,0,0,1,0,0,0,0,0,0,0
10757,HOUSEHOLD_2_515_CA_4_validation,HOUSEHOLD_2_515,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0


In [7]:
INPUT_DIR_2 = r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\CRUNCH TIME (WEEKLY AGGREGATION)\California\Household\Demand Pattern Categorization'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [8]:
len(sales_intermittent)

1493

In [9]:
len(sales_lumpy)

178

In [10]:
len(sales_erratic)

126

In [11]:
len(sales_smooth)

2391

**Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)**

In [12]:
sales_erratic

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
565,HOUSEHOLD_1_001_CA_1_validation,HOUSEHOLD_1_001,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,3,0,0,0,...,1,0,1,1,1,0,0,0,0,1
593,HOUSEHOLD_1_029_CA_1_validation,HOUSEHOLD_1_029,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,4,2,0,2,0,0,1,0
627,HOUSEHOLD_1_065_CA_1_validation,HOUSEHOLD_1_065,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,0,3,2,0,0,0,1,1
673,HOUSEHOLD_1_113_CA_1_validation,HOUSEHOLD_1_113,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,1,2,3,7,1,1,2,2,2,2
692,HOUSEHOLD_1_132_CA_1_validation,HOUSEHOLD_1_132,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,0,0,2,0,3,2,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10208,HOUSEHOLD_1_506_CA_4_validation,HOUSEHOLD_1_506,HOUSEHOLD_1,HOUSEHOLD,CA_4,CA,1,0,0,0,...,3,0,0,2,1,1,0,1,0,2
10481,HOUSEHOLD_2_239_CA_4_validation,HOUSEHOLD_2_239,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,1,1,1,...,2,1,0,3,0,0,1,2,2,1
10543,HOUSEHOLD_2_301_CA_4_validation,HOUSEHOLD_2_301,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,1,2,1,1,...,0,0,0,3,0,0,0,0,0,3
10689,HOUSEHOLD_2_447_CA_4_validation,HOUSEHOLD_2_447,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,11,...,0,6,0,0,0,0,4,0,1,1


# User-defined functions to calculate Metrics

In [13]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)


def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

# Format sales data and then merge with calendar data

In [14]:
sales_pattern = sales_erratic.copy()

In [15]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

In [16]:
dataframe_weekly

,wm_yr_wk,HOUSEHOLD_1_001_CA_1,HOUSEHOLD_1_002_CA_2,HOUSEHOLD_1_004_CA_2,HOUSEHOLD_1_008_CA_2,HOUSEHOLD_1_028_CA_2,HOUSEHOLD_1_029_CA_1,HOUSEHOLD_1_033_CA_3,HOUSEHOLD_1_043_CA_3,HOUSEHOLD_1_065_CA_1,...,HOUSEHOLD_2_447_CA_4,HOUSEHOLD_2_448_CA_3,HOUSEHOLD_2_452_CA_3,HOUSEHOLD_2_460_CA_3,HOUSEHOLD_2_498_CA_1,HOUSEHOLD_2_498_CA_3,HOUSEHOLD_2_499_CA_2,HOUSEHOLD_2_511_CA_3,HOUSEHOLD_2_512_CA_4,HOUSEHOLD_2_513_CA_1
0,11101,4.0,7.0,2.0,0.0,7.0,4.0,16.0,0.0,16.0,...,1.0,0.0,6.0,1.0,0.0,0.0,0.0,31.0,8.0,5.0
1,11102,0.0,5.0,0.0,5.0,6.0,9.0,15.0,0.0,26.0,...,0.0,0.0,8.0,4.0,0.0,0.0,0.0,16.0,1.0,0.0
2,11103,2.0,2.0,0.0,5.0,0.0,5.0,7.0,0.0,16.0,...,3.0,0.0,1.0,1.0,0.0,0.0,0.0,25.0,0.0,7.0
3,11104,0.0,1.0,0.0,4.0,1.0,6.0,12.0,0.0,14.0,...,3.0,0.0,9.0,2.0,0.0,0.0,0.0,17.0,0.0,0.0
4,11105,2.0,7.0,0.0,11.0,5.0,1.0,7.0,0.0,20.0,...,4.0,0.0,3.0,1.0,0.0,0.0,0.0,14.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,1.0,1.0,9.0,6.0,5.0,12.0,2.0,0.0,21.0,...,5.0,0.0,1.0,1.0,0.0,0.0,0.0,19.0,6.0,0.0
269,11609,0.0,1.0,5.0,6.0,13.0,12.0,2.0,0.0,28.0,...,2.0,0.0,4.0,2.0,0.0,0.0,0.0,9.0,0.0,4.0
270,11610,1.0,9.0,11.0,5.0,3.0,9.0,3.0,0.0,28.0,...,1.0,0.0,6.0,1.0,0.0,0.0,0.0,20.0,1.0,2.0
271,11611,2.0,4.0,0.0,3.0,7.0,5.0,13.0,0.0,24.0,...,5.0,0.0,6.0,1.0,0.0,0.0,0.0,4.0,1.0,0.0


In [17]:
start_train_week = 11318
end_train_week = 11610
n_pred_weeks = 2
num_train_needed = 50
validation_training_ratio = 0.95
num_id_sample_for_tuning = 200

# Run ARIMA model with best combination

In [18]:
train_data = dataframe_weekly[(dataframe_weekly.wm_yr_wk >= start_train_week) & (dataframe_weekly.wm_yr_wk <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

In [19]:
len(list_pattern_py)

126

In [20]:
best_combination = (0, 1, 2)

In [21]:
start = timer()

df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    print('Currently Running Product: %s' % product)
    print('Progressing: {0} %'.format(round(list_pattern_py.index(product) / len(list_pattern_py) * 100, 2)))
    
    train_product = train_data[product]
    test_product = test_data[product]
    
    model = ARIMA(train_product, order = best_combination)
    model_fit = model.fit()
    forecast_temp = model_fit.forecast(len(test_product))
    df_result_temp = pd.DataFrame({'Product': [product for product_count in range(len(test_product))],
                                   'Actual Data': test_product.values,
                                   'Forecast': forecast_temp.values})
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True) 
    
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running Product: HOUSEHOLD_1_001_CA_1
Progressing: 0.0 %
Currently Running Product: HOUSEHOLD_1_029_CA_1
Progressing: 0.79 %
Currently Running Product: HOUSEHOLD_1_065_CA_1
Progressing: 1.59 %
Currently Running Product: HOUSEHOLD_1_113_CA_1
Progressing: 2.38 %
Currently Running Product: HOUSEHOLD_1_132_CA_1
Progressing: 3.17 %
Currently Running Product: HOUSEHOLD_1_133_CA_1
Progressing: 3.97 %


C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base

Currently Running Product: HOUSEHOLD_1_165_CA_1
Progressing: 4.76 %
Currently Running Product: HOUSEHOLD_1_172_CA_1
Progressing: 5.56 %
Currently Running Product: HOUSEHOLD_1_212_CA_1
Progressing: 6.35 %
Currently Running Product: HOUSEHOLD_1_230_CA_1
Progressing: 7.14 %
Currently Running Product: HOUSEHOLD_1_274_CA_1
Progressing: 7.94 %
Currently Running Product: HOUSEHOLD_1_290_CA_1
Progressing: 8.73 %
Currently Running Product: HOUSEHOLD_1_325_CA_1
Progressing: 9.52 %


C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base

Currently Running Product: HOUSEHOLD_1_341_CA_1
Progressing: 10.32 %
Currently Running Product: HOUSEHOLD_1_416_CA_1
Progressing: 11.11 %
Currently Running Product: HOUSEHOLD_1_423_CA_1
Progressing: 11.9 %
Currently Running Product: HOUSEHOLD_1_452_CA_1
Progressing: 12.7 %
Currently Running Product: HOUSEHOLD_1_456_CA_1
Progressing: 13.49 %
Currently Running Product: HOUSEHOLD_1_480_CA_1
Progressing: 14.29 %
Currently Running Product: HOUSEHOLD_1_518_CA_1
Progressing: 15.08 %


...

C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base

Currently Running Product: HOUSEHOLD_1_065_CA_4
Progressing: 89.68 %
Currently Running Product: HOUSEHOLD_1_169_CA_4
Progressing: 90.48 %
Currently Running Product: HOUSEHOLD_1_172_CA_4
Progressing: 91.27 %
Currently Running Product: HOUSEHOLD_1_235_CA_4
Progressing: 92.06 %
Currently Running Product: HOUSEHOLD_1_321_CA_4
Progressing: 92.86 %
Currently Running Product: HOUSEHOLD_1_379_CA_4
Progressing: 93.65 %
Currently Running Product: HOUSEHOLD_1_416_CA_4
Progressing: 94.44 %


C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base

Currently Running Product: HOUSEHOLD_1_460_CA_4
Progressing: 95.24 %
Currently Running Product: HOUSEHOLD_1_506_CA_4
Progressing: 96.03 %
Currently Running Product: HOUSEHOLD_2_239_CA_4
Progressing: 96.83 %
Currently Running Product: HOUSEHOLD_2_301_CA_4
Progressing: 97.62 %
Currently Running Product: HOUSEHOLD_2_447_CA_4
Progressing: 98.41 %
Currently Running Product: HOUSEHOLD_2_512_CA_4
Progressing: 99.21 %
This line of code took 0.08091130833333343 minutes


C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


# Calculate metrics for optimal combination

In [22]:
# Start calculating metrics 
start = timer()

df_arima_metrics = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.0034298983333333885 minutes


In [23]:
df_arima_metrics

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_001_CA_1,0.526930,35.1286,35.9054,27.0175,5.269296e-01,7.264603e-01
1,HOUSEHOLD_1_002_CA_2,inf,9.3415,8.7786,9.3415,3.736608e-01,4.557642e-01
2,HOUSEHOLD_1_004_CA_2,inf,inf,200.0000,NaN,8.212012e+00,8.216736e+00
3,HOUSEHOLD_1_008_CA_2,inf,68.2190,50.8681,68.2190,2.046569e+00,2.046569e+00
4,HOUSEHOLD_1_028_CA_2,0.836608,27.8869,25.5432,32.6022,1.673216e+00,2.163459e+00
...,...,...,...,...,...,...,...
121,HOUSEHOLD_2_498_CA_3,inf,inf,200.0000,NaN,8.961502e-02,9.236135e-02
122,HOUSEHOLD_2_499_CA_2,inf,inf,200.0000,NaN,4.946398e-20,5.040407e-20
123,HOUSEHOLD_2_511_CA_3,1.884299,144.9461,87.6977,186.8094,9.421494e+00,9.723526e+00
124,HOUSEHOLD_2_512_CA_4,0.462825,61.7100,70.5488,116.4460,1.851300e+00,1.856158e+00


In [24]:
df_arima_metrics.MAPE.mean()

71.86077244897959

In [25]:
df_arima_metrics.RMSE.mean()

3.7964595305974274

In [26]:
df_arima_metrics.to_csv('ARIMA_Erratic_Forecast_Metrics.csv')